# Automatic Fact-checking project using pandas and ScikitLearn

### Travail réalisé par : Bouali Mohammed-Amin, Oussama Nassim Sehout, Chahinez Benallal, Abdellah Choukri 



####  Sommaire du travail :
####   . Chargement du jeu de données
####   . Prétraitement du texte 
###   . Conversion des textes aux valeurs numériques
###   . Division du jeu de données en données d'entraînement et données de test
###   . Entraînement et prédiction en utilisant des modèles de classification

## Etape 1 : Chargement du jeu de données  
 On a généré trois fichiers csv, un avec valeurs true, un autre avec valeurs false, et un dernier avec valeurs mixture à partir du site ClaimsKg, après on les concatène pour avoir notre jeu de données.

In [1]:
import pandas as pd
import random as random
import glob, os

#concatener les fichiers en python  et lecture du fichier :
df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "./DataSet/*.csv"))))

row, col = df.shape
print("Size of the dataframe :" + str(df.size) + " ("+str(row)+"*"+str(col)+")")

df.head()

df


Size of the dataframe :202328 (14452*14)


,id,text,date,truthRating,ratingName,author,headline,named_entities_claim,named_entities_article,keywords,source,sourceURL,link,language
0,http://data.gesis.org/claimskg/claim_review/36...,'There will be no public funding for abortion ...,2010-03-21,3,True,Bart Stupak,Stupak revises abortion stance on health care ...,"Abortion rights,Barack Obama,Bart Stupak,Ben N...",abortion,"Abortion,Health Care",politifact,http://www.politifact.com,http://www.politifact.com/truth-o-meter/statem...,English
1,http://data.gesis.org/claimskg/claim_review/e6...,Central Health 'is the only hospital district ...,2011-03-15,3,True,Wayne Christian,State Rep. Wayne Christian says Central Health...,"Austin American-Statesman,Harris County Hospit...",NaN,Abortion,politifact,http://www.politifact.com,http://www.politifact.com/texas/statements/201...,English
2,http://data.gesis.org/claimskg/claim_review/e0...,Says most of Perry's chiefs of staff have been...,2010-08-14,3,True,Bill White,Bill White says most of Gov. Rick Perry's chie...,"AT&T,Bill Clements,Bill White,Bracewell & Giul...",NaN,Ethics,politifact,http://www.politifact.com,http://www.politifact.com/texas/statements/201...,English
3,http://data.gesis.org/claimskg/claim_review/48...,Says 'as Co-Chair of the Joint Ways & Means Co...,2012-09-28,3,True,Mary Nolan,Did Mary Nolan secure funding for Milwaukie br...,"Carolyn Tomei,Dave Hunt,Fetsch,Jeff Merkley,Ka...",Portland-Milwaukie Light Rail project,"State Budget,State Finances,Transportation",politifact,http://www.politifact.com,http://www.politifact.com/oregon/statements/20...,English
4,http://data.gesis.org/claimskg/claim_review/80...,Says Gary Farmer’s claim that he 'received an ...,2016-07-08,3,True,Jim Waldman,Florida Senate candidate never actually receiv...,"Gary Farmer,Gwyndolen Clarke-Reed,Jim Waldman,...",Gary Farmer,Guns,politifact,http://www.politifact.com,http://www.politifact.com/florida/statements/2...,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,http://data.gesis.org/claimskg/claim_review/cd...,Kevin’s lovable yet geeky sidekick in TV’s “Th...,Unknown,1,False,Unknown,Marilyn Manson and The Wonder Years,"Alice Cooper,Behind the Green Door,Bewitched,B...","Marilyn Manson,Wonder Years","Artists, ASP Article, music, Radio & TV, Telev...",snopes,http://www.snopes.com,https://www.snopes.com/fact-check/wondering-ab...,English
9996,http://data.gesis.org/claimskg/claim_review/7f...,E-mail reproduces George Carlin’s list of New ...,Unknown,1,False,Unknown,George Carlin New Rules for 2006,"George Carlin,HBO,Real Time with Bill Maher,th...",George Carlin,ASP Article,snopes,http://www.snopes.com,https://www.snopes.com/fact-check/new-rules-fo...,English
9997,http://data.gesis.org/claimskg/claim_review/72...,'Obamacare Medical Codes Confirm: Execution by...,2013-11-23,1,False,Bloggers,Bloggers say Obamacare coding system could ush...,"American Medical Association,Centers for Disea...",Medical Codes,"Health Care,Legal Issues,Public Health",politifact,http://www.politifact.com,http://www.politifact.com/truth-o-meter/statem...,English
9998,http://data.gesis.org/claimskg/claim_review/7c...,Wrestler John Cena died in a car accident in J...,Unknown,1,False,Unknown,John Cena Death Hoax,"Facebook,Interstate 80,John Cena,WWE,fake news...",John Cena,"death hoax, john cena, john cena dead",snopes,http://www.snopes.com,https://www.snopes.com/fact-check/john-cena-de...,English


### Etape 2 : Prétraitement du texte
 On a fait les prétraitements suivants :
     . Transformation du texte en miniscule
     . Suppression des espaces
     . Enlever les ponctuations 
     . Elimination des stopwords
     . Remplacement des mots de négation par le mot 'not'
     . Suppression des caractères non ASCII
     . Lemmatization 
     . Correction orthographique ##TODO

In [2]:
#commencement des prétraitement :

#import nécessaire :
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
nltk.download('stopwords') 
nltk.download('punkt')
from nltk.corpus import stopwords
import unicodedata
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from collections import Counter
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
import inflect




#La j'ai mis que la colonne text en lower mais je crois qu'on devra tous les mettre par la suite
df['text'] = df['text'].str.lower()


#White spaces removal
df['text'] = df['text'].str.strip()

#La j'ai mis que la colonne text en lower mais je crois qu'on devra tous les mettre par la suite
df['keywords'] = df['keywords'].str.lower()


#White spaces removal
df['keywords'] = df['keywords'].str.strip()
df['keywords'] = df['keywords'].fillna('nan')

df['named_entities_claim']=df['named_entities_claim'].str.strip()
df['named_entities_claim']=df['named_entities_claim'].str.lower()
df['named_entities_claim'] = df['named_entities_claim'].fillna('nan')

# df['headline']=df['headline'].str.strip()
# df['headline']=df['headline'].str.lower()
# df['headline'] = df['headline'].fillna('nan')


#Elimination des stopWord :
#input_str = "NLTK is a leading platform for building Python programs to work with human language data."
stop_words = set(stopwords.words("english"))
#La liste des stopwords en Anglais
negation ={'haven''t','cannot',"doesn't","shouldn't","needn't","shant't","weren't","hasn't", "wasn't","didn't", "aren't",'not', "mightn't", "mustn't", 'no',  "wouldn't", "mightn't", "won't",  "needn't", "wasn't", "wouldn't",  "isn't", "doesn't", "weren't", "isn't", "hasn't", "hadn't", "don't", "hadn't","couldn't"}

#La liste des stopwords de négation en Anglais  
from nltk.tokenize import TweetTokenizer
a= df['text'].str.replace("’","'") #Extraire toutes les entrées de la colonne text
pat = r'\b(?:{})\b'.format('|'.join(negation))

a=a.str.replace(pat,'not')
pat='\w*\d\w*'
text_without_stopwords=[] #Une liste dont on va affecter les textes après l'élimination des stop words
tk=TweetTokenizer()
for text in a.iteritems(): #On parcourt toutes les lignes
 tokens = tk.tokenize(str(text[1]))
 result = [i for i in tokens if not i in stop_words-negation]
 splitor=" "
 concatinated = splitor.join(result) #concatiner les tokens
 text_without_stopwords.append(concatinated)

df['text']=text_without_stopwords

a= df['named_entities_claim'].str.replace("’","'") #Extraire toutes les entrées de la colonne text
text_without_stopwords=[] #Une liste dont on va affecter les textes après l'élimination des stop words
tk=TweetTokenizer()
for text in a.iteritems(): #On parcourt toutes les lignes
 tokens = tk.tokenize(str(text[1]))
 result = [i for i in tokens if not i in stop_words-negation]
 splitor=" "
 concatinated = splitor.join(result) #concatiner les tokens
 text_without_stopwords.append(concatinated)

df['named_entities_claim']=text_without_stopwords


#The following code removes this set of symbols [!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]:
text_without_punctuation=[] # pour y mettre notre résultat
a= df['text'] #Extraire toutes les entrées de la colonne text deja traiter pour faire la suite
tk=TweetTokenizer()
for text in a.iteritems(): #On parcourt toutes les lignes
    result = re.sub('[!”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', '', str(text[1]) )
    splitor="" #séparateur de mots
    concatinated = splitor.join(result) #concatiner les résultats
    text_without_punctuation.append(concatinated)

df['text']=text_without_punctuation


#The following code removes non ASCII characters :
text_without_ascii=[] # pour y mettre notre résultat
a= df['text'] #Extraire toutes les entrées de la colonne text deja traiter pour faire la suite
tk=TweetTokenizer()
for text in a.iteritems(): #On parcourt toutes les lignes
    text = unicodedata.normalize('NFKD', str(text[1]) ).encode("ascii", "ignore").decode("utf-8", 'ignore')
    splitor="" #séparateur de mots
    concatinated = splitor.join(text)
    text_without_ascii.append(concatinated)

df['text']=text_without_ascii

def nombreversmot(text):
    if text.isdigit():
        return p.number_to_words(text)
    else: return text
print("---------- Avant Transformation des numériques en mots -------------------")
#Transformation des numériques en mots
numbertransf=[] # pour y mettre notre résultat
p = inflect.engine()
a= df['text'] #Extraire toutes les entrées de la colonne text deja traiter pour faire la suite
tk=TweetTokenizer()
for text in a.iteritems():
     tokens = tk.tokenize(str(text[1]))
     result = [nombreversmot(i) for i in tokens]
     splitor=" "
     concatinated = splitor.join(result) #concatiner les tokens
     numbertransf.append(concatinated)

df['text']=numbertransf
df['text'] = df['text'].str.strip()

numbertransf=[] # pour y mettre notre résultat
p = inflect.engine()
a= df['named_entities_claim'] #Extraire toutes les entrées de la colonne text deja traiter pour faire la suite
tk=TweetTokenizer()
for text in a.iteritems():
     tokens = tk.tokenize(str(text[1]))
     result = [nombreversmot(i) for i in tokens]
     splitor=" "
     concatinated = splitor.join(result) #concatiner les tokens
     numbertransf.append(concatinated)

df['named_entities_claim']=numbertransf
df['named_entities_claim'] = df['named_entities_claim'].str.strip()
print(df['named_entities_claim'].head(10))
print("---------- Après Transformation des numériques en mots -------------------")


#Lemmatization :
lemmatizer = WordNetLemmatizer()
def nltk2wn_tag(nltk_tag):
  if nltk_tag.startswith('J'):
    return wordnet.ADJ
  elif nltk_tag.startswith('V'):
    return wordnet.VERB
  elif nltk_tag.startswith('N'):
    return wordnet.NOUN
  elif nltk_tag.startswith('R'):
    return wordnet.ADV
  else:                    
    return None
def lemmatize_sentence(sentence):
  nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))    
  wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
  res_words = []
  for word, tag in wn_tagged:
    if tag is None:                        
      res_words.append(word)
    else:
      res_words.append(lemmatizer.lemmatize(word, tag))
  return " ".join(res_words)

text_lemmatizer=[] # pour y mettre notre résultat
a= df['text'] #Extraire toutes les entrées de la colonne text deja traiter pour faire la suite
tk=TweetTokenizer()
for ligne in a.iteritems(): #On parcourt toutes les lignes
    newList=[]
    newList.append(lemmatize_sentence(ligne[1])) #lemmatisation
    splitor=" " #séparateur de mots
    concatinated = splitor.join(newList)
    text_lemmatizer.append(concatinated)
df['text']=text_lemmatizer
# #Lemmatization :
# lemmatizer = WordNetLemmatizer()
# text_lemmatizer=[] # pour y mettre notre résultat
# a= df['keywords'] #Extraire toutes les entrées de la colonne text deja traiter pour faire la suite
# tk=TweetTokenizer()
# for ligne in a.iteritems(): #On parcourt toutes les lignes 
#     wordList = re.sub("[^\w]", " ",  ligne[1]).split() # on parcourt tout les mot de la ligne
#     newList=[]
#     for word in wordList:
#         if not(word.isdigit()): # si c'est  mot et non un nombre
#             newList.append(lemmatizer.lemmatize(word, pos = 'v')) #lemmatisation
#     splitor=" " #séparateur de mots
#     concatinated = splitor.join(newList)
#     text_lemmatizer.append(concatinated)

# df['keywords']=text_lemmatizer

#Lemmatization :
text_lemmatizer=[] # pour y mettre notre résultat
a= df['named_entities_claim'] #Extraire toutes les entrées de la colonne text deja traiter pour faire la suite
tk=TweetTokenizer()
for ligne in a.iteritems(): #On parcourt toutes les lignes
    newList=[]
    newList.append(lemmatize_sentence(ligne[1])) #lemmatisation
    splitor=" " #séparateur de mots
    concatinated = splitor.join(newList)
    text_lemmatizer.append(concatinated)
df['named_entities_claim']=text_lemmatizer

# #Lemmatization :
# text_lemmatizer=[] # pour y mettre notre résultat
# a= df['headline'] #Extraire toutes les entrées de la colonne text deja traiter pour faire la suite
# tk=TweetTokenizer()
# for ligne in a.iteritems(): #On parcourt toutes les lignes
#     newList=[]
#     newList.append(lemmatize_sentence(ligne[1])) #lemmatisation
#     splitor=" " #séparateur de mots
#     concatinated = splitor.join(newList)
#     text_lemmatizer.append(concatinated)
# df['headline']=text_lemmatizer



# #suppression des common word:
# word_counter  = Counter()
# for sentence in df["text"].values:
#     for word in sentence.split():
#         word_counter[word] += 1
# most = word_counter.most_common(10)
# print("most common word"+str(most))
# print("Suppression of the common word de nos artiles : ")
# most_word = set([w for (w, wc) in most])
# def delmost_word(sentence):
#     return " ".join([word for word in str(sentence).split() if word not in most_word])
# df["text"] = df["text"].apply(delmost_word)
# df["text"].head()

df

#Fin des prétraitement.

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/depinfo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/depinfo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/depinfo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/depinfo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


---------- Avant Transformation des numériques en mots -------------------
0    abortion rights , barack obama , bart stupak ,...
1    austin american-statesman , harris county hosp...
2    & , bill clements , bill white , bracewell & g...
3    carolyn tomei , dave hunt , fetsch , jeff merk...
4    gary farmer , gwyndolen clarke-reed , jim wald...
5    democratic governors association , john chafee...
6    114th congress , two thousand and sixteen gene...
7    two thousand and sixteen presidential campaign...
8    cuba , facebook , guantanamo bay , guantánamo ...
9    afghanistan , christmas decoration , japan , l...
Name: named_entities_claim, dtype: object
---------- Après Transformation des numériques en mots -------------------


,id,text,date,truthRating,ratingName,author,headline,named_entities_claim,named_entities_article,keywords,source,sourceURL,link,language
0,http://data.gesis.org/claimskg/claim_review/36...,' not public funding abortion legislation ',2010-03-21,3,True,Bart Stupak,Stupak revises abortion stance on health care ...,"abortion right , barack obama , bart stupak , ...",abortion,"abortion,health care",politifact,http://www.politifact.com,http://www.politifact.com/truth-o-meter/statem...,English
1,http://data.gesis.org/claimskg/claim_review/e6...,central health ' hospital district texas spend...,2011-03-15,3,True,Wayne Christian,State Rep. Wayne Christian says Central Health...,"austin american-statesman , harris county hosp...",NaN,abortion,politifact,http://www.politifact.com,http://www.politifact.com/texas/statements/201...,English
2,http://data.gesis.org/claimskg/claim_review/e0...,say perry 's chiefs staff lobbyist,2010-08-14,3,True,Bill White,Bill White says most of Gov. Rick Perry's chie...,"& , bill clements , bill white , bracewell & g...",NaN,ethics,politifact,http://www.politifact.com,http://www.politifact.com/texas/statements/201...,English
3,http://data.gesis.org/claimskg/claim_review/48...,say ' cochair joint way mean committee secure ...,2012-09-28,3,True,Mary Nolan,Did Mary Nolan secure funding for Milwaukie br...,"carolyn tomei , dave hunt , fetsch , jeff merk...",Portland-Milwaukie Light Rail project,"state budget,state finances,transportation",politifact,http://www.politifact.com,http://www.politifact.com/oregon/statements/20...,English
4,http://data.gesis.org/claimskg/claim_review/80...,say gary farmer 's claim ' receive ' ' nra ' '...,2016-07-08,3,True,Jim Waldman,Florida Senate candidate never actually receiv...,"gary farmer , gwyndolen clarke-reed , jim wald...",Gary Farmer,guns,politifact,http://www.politifact.com,http://www.politifact.com/florida/statements/2...,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,http://data.gesis.org/claimskg/claim_review/cd...,kevin 's lovable yet geeky sidekick tv 's wond...,Unknown,1,False,Unknown,Marilyn Manson and The Wonder Years,"alice cooper , behind green door , bewitch , b...","Marilyn Manson,Wonder Years","artists, asp article, music, radio & tv, telev...",snopes,http://www.snopes.com,https://www.snopes.com/fact-check/wondering-ab...,English
9996,http://data.gesis.org/claimskg/claim_review/7f...,email reproduces george carlin 's list new rul...,Unknown,1,False,Unknown,George Carlin New Rules for 2006,"george carlin , hbo , real time bill maher , i...",George Carlin,asp article,snopes,http://www.snopes.com,https://www.snopes.com/fact-check/new-rules-fo...,English
9997,http://data.gesis.org/claimskg/claim_review/72...,' obamacare medical code confirm execution beh...,2013-11-23,1,False,Bloggers,Bloggers say Obamacare coding system could ush...,"american medical association , center disease ...",Medical Codes,"health care,legal issues,public health",politifact,http://www.politifact.com,http://www.politifact.com/truth-o-meter/statem...,English
9998,http://data.gesis.org/claimskg/claim_review/7c...,wrestler john cena die car accident july two t...,Unknown,1,False,Unknown,John Cena Death Hoax,"facebook , interstate eighty , john cena , wwe...",John Cena,"death hoax, john cena, john cena dead",snopes,http://www.snopes.com,https://www.snopes.com/fact-check/john-cena-de...,English


### Etape 3 : Conversion des textes en valeurs numériques
 On transforme nos données de textes en valeurs numériques, en utilisant des LabelEncoder sur les colonnes qu'on a  sauf la colonne texte et la colonne keyword, dont on a essayé Tf-IDF pour les transformer. 

In [3]:
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer




#np.random.seed(500) #utilisé pour avoir le même résultat à chaque exécution 
#On crée des variable LabelEncoder qui vont servir à transférer nos données en valeurs numériques
l1=LabelEncoder()
l2=LabelEncoder()
l3=LabelEncoder()
l4=LabelEncoder()
l5=LabelEncoder()
l6=LabelEncoder()
l7=LabelEncoder()
l8=LabelEncoder()
l9=LabelEncoder()
l10=LabelEncoder()
l11=LabelEncoder()
l12=LabelEncoder()
l13=LabelEncoder()
df=df.applymap(str) #on transforme tous nous données en String car y'avais des entrées qui ont une combinaison du string et float
#On applique la mesure TF-IDF sur la colonne text et la colonne keywords
# Tfidf_vect = TfidfVectorizer(max_features=40000,min_df=0.03, max_df=0.7)
# tfidfx=Tfidf_vect.fit_transform(df['named_entities_claim'])
# df1 = pd.DataFrame(tfidfx.toarray(), columns=Tfidf_vect.get_feature_names())
# Tfidf_vect = TfidfVectorizer(max_features=4000)
# print(df['named_entities_claim'])
cd = CountVectorizer(min_df=0.1)
cd.fit(df["text"])
tfidfx=cd.transform(df['text'])
df2 = pd.DataFrame(tfidfx.toarray(), columns=cd.get_feature_names())
print(df2)
cd = CountVectorizer(min_df=0.1)
cd.fit(df["named_entities_claim"])
tfidfx=cd.transform(df['text'])
df1 = pd.DataFrame(tfidfx.toarray(), columns=cd.get_feature_names())
print(df1)

# cd = CountVectorizer(min_df=0.1)
# cd.fit(df["headline"])
# tfidfx=cd.transform(df['text'])
# df3 = pd.DataFrame(tfidfx.toarray(), columns=cd.get_feature_names())
# print(df3)

#on transfère toutes les valeurs des colonnes qu'on va utiliser en valeurs numériques 
df['id']=l1.fit_transform(df['id'])
df['text']=l3.fit_transform(df['text'])
df['date']=l3.fit_transform(df['date'])
df['author']=l5.fit_transform(df['author'])
df['headline']=l6.fit_transform(df['headline'])
# df['named_entities_claim']=l7.fit_transform(df['named_entities_claim'])
df['named_entities_article']=l8.fit_transform(df['named_entities_article'])
df['source']=l10.fit_transform(df['source'])
df['sourceURL']=l11.fit_transform(df['sourceURL'])
df['link']=l12.fit_transform(df['link'])
df['language']=l13.fit_transform(df['language'])
df['ratingName']=l13.fit_transform(df['ratingName'])


df= pd.concat([df, df1], axis=1,join_axes=[df.index])
df=pd.concat([df, df2], axis=1,join_axes=[df.index])
df1=pd.concat([df1, df2], axis=1,join_axes=[df1.index])
# df=pd.concat([df, df3], axis=1,join_axes=[df.index])







       and  say  thousand  two
0        0    0         0    0
1        0    0         0    0
2        0    1         0    0
3        0    1         0    0
4        0    1         0    0
...    ...  ...       ...  ...
14447    0    0         0    0
14448    1    0         1    1
14449    0    0         0    0
14450    1    0         1    1
14451    0    0         0    0

[14452 rows x 4 columns]
       barack  donald  facebook  house  http  internet  john  national  new  \
0           0       0         0      0     0         0     0         0    0   
1           0       0         0      0     0         0     0         0    0   
2           0       0         0      0     0         0     0         0    0   
3           0       0         0      0     0         0     0         0    0   
4           0       0         0      0     0         0     0         0    0   
...       ...     ...       ...    ...   ...       ...   ...       ...  ...   
14447       0       0         0      0     0     

/home/depinfo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:64: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
/home/depinfo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:65: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
/home/depinfo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:66: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.


### Etape 4 : Dévision du jeu de données en données d'entraînement et données de test
On mélange le dataframe qu'on a et après on sélectionne 80% des données pour l'entraînement et 20% pour le test, ensuite on sélectionne les colonnes features dont on s'intéresse lors du classification, et on essaye plusieurs combinaisons de colonnes pour arriver à une meilleure accuracy (#TODO).

In [4]:
from sklearn.utils import shuffle
import numpy as np
##### 
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn import metrics


X_train, X_test ,y_train, y_test = train_test_split(df,df["truthRating"], test_size=0.2, random_state=int(time.time()))


gnb = GaussianNB()

used_features =[
#     "author",
#     "sourceURL"
    
]
for col in df1.columns :
    used_features.append(col)
for col in df2.columns :
    used_features.append(col)
# for col in df3.columns :
#     used_features.append(col)

gnb.fit(
    X_train[used_features].values,
    y_train
)
mnb=MultinomialNB()
mnb.fit(X_train[used_features].values,
    y_train)

y_pred = gnb.predict(X_test[used_features])
mnbpred=mnb.predict(X_test[used_features])
# print("Number of mislabeled points out of a total {} points : {}, performance {:05.2f}%"
#       .format(
#           X_test.shape[0],
#           (X_test["ratingName"] != y_pred).sum(),
#           100*(1-(X_test["ratingName"] != y_pred).sum()/X_test.shape[0])
# ))
# print(df['ratingName'])
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Accuracy:",metrics.accuracy_score(y_test, mnbpred))
print(len(used_features))##

Accuracy: 0.3331027326184711
Accuracy: 0.6841923209961951
29


### Etape 5 :  Entraînement et prédiction en utilisant des modèles de 


classification
On passe les features et targets des données d'entraînement à nos classifiers (#TODO tester plusieurs classifieurs) et après on lui laisse prédire les valeurs des données de test (soit vrai, faux ou mixture).

In [5]:
# Begin by importing all necessary libraries
# import pandas as pd
# import time
# from sklearn.utils import shuffle
# import numpy as np
# ##### 
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

# #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=int(time.time()))
# label = df["ratingName"]
# features=zip(df["date"],df["author"],df["named_entities_claim"],df["source"],df["sourceURL"])
# # dt = df["date"]
# # src = df["sourceURL"]
# # features=zip(dt,src)

# pip

# #print(df["author"])  
# print(features)